In [1]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import wandb

In [2]:
class TransliterationDataset(Dataset):
    def __init__(self, file_path, source_vocab=None, target_vocab=None, max_length=32, create_vocab=False):
        # Attempt to read tab-separated file containing native and romanized text
        try:
            data = pd.read_csv(file_path, sep='\t', header=None,
                               names=['native', 'roman', 'freq'],
                               usecols=[0, 1], dtype=str)
            print(f"Loaded {len(data)} records from {file_path}")

            # Replace missing entries with empty strings
            data['native'] = data['native'].fillna('')
            data['roman'] = data['roman'].fillna('')

            # Create input-output text pairs for transliteration
            self.data_pairs = list(zip(data['roman'], data['native']))
            print(f"Example pairs: {self.data_pairs[:2]}")
        except Exception as err:
            print(f"Failed to load data: {err}")
            self.data_pairs = [('', '')]  # Fallback entry

        self.max_length = max_length

        # Initialize or use existing vocabulary mappings
        if create_vocab:
            self.source_vocab = {'<pad>': 0, '<unk>': 1, '<sos>':2, '<eos>':3}
            self.target_vocab = {'<pad>': 0, '<unk>': 1, '<sos>':2, '<eos>':3}
            self._generate_vocab()
        else:
            self.source_vocab, self.target_vocab = source_vocab, target_vocab
            if '<eos>' not in self.source_vocab:
                self.source_vocab['<eos>'] = len(self.source_vocab)
            if '<eos>' not in self.target_vocab:
                self.target_vocab['<eos>'] = len(self.target_vocab)

    def _generate_vocab(self):
        # Create vocabularies at the character level from the dataset
        for src_text, tgt_text in self.data_pairs:
            for char in src_text:
                if char not in self.source_vocab:
                    self.source_vocab[char] = len(self.source_vocab)
            for char in tgt_text:
                if char not in self.target_vocab:
                    self.target_vocab[char] = len(self.target_vocab)
        print(f"Vocab sizes — Source: {len(self.source_vocab)}, Target: {len(self.target_vocab)}")

    def __len__(self):
        return len(self.data_pairs)

    def __getitem__(self, idx):
        src_seq, tgt_seq = self.data_pairs[idx]
        
        # Convert characters to indices with bounds checking
        src_idxs = [self.source_vocab['<sos>']]  # Start with SOS token
        for ch in src_seq:
            idx = self.source_vocab.get(ch, self.source_vocab['<unk>'])
            if idx >= len(self.source_vocab):
                idx = self.source_vocab['<unk>']  # Safety check
            src_idxs.append(idx)
            
        tgt_idxs = [self.target_vocab['<sos>']]  # Start with SOS token
        for ch in tgt_seq:
            idx = self.target_vocab.get(ch, self.target_vocab['<unk>'])
            if idx >= len(self.target_vocab):
                idx = self.target_vocab['<unk>']  # Safety check
            tgt_idxs.append(idx)
        
        # Add end-of-sequence token
        src_idxs.append(self.source_vocab['<eos>'])
        tgt_idxs.append(self.target_vocab['<eos>'])
        
        # Add padding
        pad_src = [self.source_vocab['<pad>']] * max(0, self.max_length - len(src_idxs))
        pad_tgt = [self.target_vocab['<pad>']] * max(0, self.max_length - len(tgt_idxs))
        
        # Truncate if necessary and convert to tensor
        src_idxs = (src_idxs + pad_src)[:self.max_length]
        tgt_idxs = (tgt_idxs + pad_tgt)[:self.max_length]
        
        # Make sure padding index is valid
        assert self.source_vocab['<pad>'] < len(self.source_vocab), "Padding index out of bounds for source vocab"
        assert self.target_vocab['<pad>'] < len(self.target_vocab), "Padding index out of bounds for target vocab"
        
        return torch.tensor(src_idxs, dtype=torch.long), torch.tensor(tgt_idxs, dtype=torch.long)

In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_vocab_size, embedding_size, hidden_size, num_layers, dropout=0.0, cell_type='GRU'):
        super().__init__()
        self.embedding = nn.Embedding(input_vocab_size, embedding_size, padding_idx=0)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        
        # Choose appropriate RNN cell
        if cell_type == 'RNN':
            rnn_cell = nn.RNN
        elif cell_type == 'GRU':
            rnn_cell = nn.GRU
        elif cell_type == 'LSTM':
            rnn_cell = nn.LSTM
        else:
            raise ValueError(f"RNN type not recognized: {cell_type}")
        
        self.rnn = rnn_cell(
            embedding_size, 
            hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
    
    def forward(self, x):
        """
        Args:
            x: input sequence [batch_size, seq_len]
        Returns:
            outputs: encoder outputs [batch_size, seq_len, hidden_size]
            hidden: final hidden state [num_layers, batch_size, hidden_size]
        """
        # Apply embedding layer: [batch_size, seq_len] -> [batch_size, seq_len, embedding_size]
        embedded = self.embedding(x)
        
        # Pass through RNN
        outputs, hidden = self.rnn(embedded)
        
        return outputs, hidden

class DecoderRNN(nn.Module):
    def __init__(self, output_vocab_size, embedding_size, hidden_size, num_layers, dropout=0.0, cell_type='GRU'):
        super().__init__()
        self.output_vocab_size = output_vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        
        self.embedding = nn.Embedding(output_vocab_size, embedding_size, padding_idx=0)
        
        # Choose appropriate RNN cell
        if cell_type == 'RNN':
            rnn_cell = nn.RNN
        elif cell_type == 'GRU':
            rnn_cell = nn.GRU
        elif cell_type == 'LSTM':
            rnn_cell = nn.LSTM
        else:
            raise ValueError(f"RNN type not recognized: {cell_type}")
        
        self.rnn = rnn_cell(
            embedding_size, 
            hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        
        # Linear layer to produce output logits
        self.output_projection = nn.Linear(hidden_size, output_vocab_size)
    
    def forward(self, input_token, last_hidden):
        """
        Args:
            input_token: input token indices [batch_size, 1]
            last_hidden: previous decoder hidden state
        Returns:
            output: output logits [batch_size, output_vocab_size]
            hidden: new hidden state
        """
        # Get embedding for input token [batch_size, 1, embedding_size]
        embedded = self.embedding(input_token)
        
        # Pass through RNN
        output, hidden = self.rnn(embedded, last_hidden)
        
        # Project to vocabulary space [batch_size, output_vocab_size]
        output = self.output_projection(output.squeeze(1))
        
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, config, input_vocab_size, output_vocab_size):
        super().__init__()

        # Ensure vocabulary sizes are positive
        assert input_vocab_size > 0, f"Source vocab size invalid: {input_vocab_size}"
        assert output_vocab_size > 0, f"Target vocab size invalid: {output_vocab_size}"

        # Hyperparameter setup
        self.embedding_size = config.embed_dim
        self.hidden_size = config.hidden_dim
        self.rnn_type = config.cell_type
        self.encoder_layers = config.enc_layers
        self.decoder_layers = config.dec_layers
        
        # Initialize encoder and decoder
        self.encoder = EncoderRNN(
            input_vocab_size, 
            config.embed_dim, 
            config.hidden_dim, 
            config.enc_layers, 
            config.dropout, 
            config.cell_type
        )
        
        self.decoder = DecoderRNN(
            output_vocab_size, 
            config.embed_dim, 
            config.hidden_dim, 
            config.dec_layers, 
            config.dropout, 
            config.cell_type
        )
        
        # For accessing embeddings in training/inference
        self.input_embedding = self.encoder.embedding
        self.output_embedding = self.decoder.embedding

        print(f"Initialized model without attention: {self.rnn_type}, Encoder layers: {self.encoder_layers}, "
              f"Decoder layers: {self.decoder_layers}, Embedding: {self.embedding_size}, Hidden: {self.hidden_size}")

    def forward(self, source_seq, target_seq):
        """
        Forward pass through the entire seq2seq model.
        
        Args:
            source_seq: source sequence [batch_size, src_len]
            target_seq: target sequence [batch_size, tgt_len]
        
        Returns:
            outputs: output logits [batch_size, tgt_len-1, output_vocab_size]
        """
        batch_size, src_len = source_seq.size()
        tgt_len = target_seq.size(1)
        device = source_seq.device

        # Index range check and clamping
        if source_seq.max() >= self.input_embedding.num_embeddings:
            print("Warning: Source index out of bounds")
            source_seq = torch.clamp(source_seq, 0, self.input_embedding.num_embeddings - 1)
        if target_seq.max() >= self.output_embedding.num_embeddings:
            print("Warning: Target index out of bounds")
            target_seq = torch.clamp(target_seq, 0, self.output_embedding.num_embeddings - 1)

        try:
            # Encode source sequence
            _, encoder_hidden = self.encoder(source_seq)
            
            # Initialize decoder hidden state with encoder final state
            decoder_hidden = encoder_hidden
            
            # Teacher forcing: feed target as the next input
            decoder_input = target_seq[:, :-1]  # Input: from <sos> to the second-to-last token
            # Store all decoder outputs
            outputs = torch.zeros(batch_size, tgt_len-1, self.decoder.output_vocab_size, device=device)
            
            # Process one token at a time
            for t in range(tgt_len-1):
                # Get token for current timestep
                current_input = target_seq[:, t].unsqueeze(1)
                
                # Forward through decoder
                decoder_output, decoder_hidden = self.decoder(
                    current_input,
                    decoder_hidden
                )
                
                # Store prediction
                outputs[:, t, :] = decoder_output
            
            return outputs
            
        except Exception as err:
            print(f"Forward pass exception: {err}")
            return torch.zeros(batch_size, tgt_len-1, self.decoder.output_vocab_size, device=device)

    def inference(self, source_seq, max_length=50, beam_size=1):
        """
        Generate transliteration using beam search.
        
        Args:
            source_seq: source sequence [batch_size, src_len]
            max_length: maximum length of generated sequence
            beam_size: beam size for beam search
            
        Returns:
            generated_seq: generated sequence [batch_size, max_length]
        """
        batch_size = source_seq.size(0)
        device = source_seq.device
        
        # Encode source sequence
        _, encoder_hidden = self.encoder(source_seq)
        
        if beam_size == 1:
            # Greedy search
            decoder_hidden = encoder_hidden
            decoder_input = torch.tensor([[2]], device=device).repeat(batch_size, 1)  # Start with <sos> token
            generated_tokens = torch.zeros(batch_size, max_length, dtype=torch.long, device=device)
            
            for t in range(max_length):
                # Forward through decoder
                decoder_output, decoder_hidden = self.decoder(
                    decoder_input,
                    decoder_hidden
                )
                
                # Get most likely token
                _, topi = decoder_output.topk(1)
                decoder_input = topi.view(batch_size, 1)
                
                # Save generated token
                generated_tokens[:, t] = decoder_input.squeeze(1)
                
                # Stop if all sequences generated <eos>
                if (decoder_input == 3).all():  # 3 is <eos> token index
                    break
            
            return generated_tokens
        else:
            # Implement beam search here (omitted for brevity)
            # For now, return greedy search result
            return self.inference(source_seq, max_length, beam_size=1)

In [4]:
def compute_accuracy(logits, target, pad_idx=0):
    """
    Computes accuracy excluding padding tokens.
    """
    preds = logits.argmax(dim=-1)
    mask = target != pad_idx
    correct = (preds == target) & mask
    acc = correct.sum().item() / max(mask.sum().item(), 1)  # Avoid division by zero
    return acc

In [5]:
def run_epoch(model, data_loader, loss_fn, optim, device):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    total_batches = len(data_loader)
    processed_batches = 0

    for i, (inputs, targets) in enumerate(data_loader):
        try:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Check for out-of-vocabulary token indices
            if inputs.max().item() >= model.input_embedding.num_embeddings or \
               targets.max().item() >= model.output_embedding.num_embeddings:
                print(f"Skipping batch {i}/{total_batches} - "
                      f"Input max: {inputs.max().item()}, Target max: {targets.max().item()}, "
                      f"Input vocab size: {model.input_embedding.num_embeddings}, "
                      f"Target vocab size: {model.output_embedding.num_embeddings}")
                continue

            # Reset gradients
            optim.zero_grad()

            # Model inference
            output_logits = model(inputs, targets)

            # Flatten predictions and targets for loss function
            pred_flat = output_logits.reshape(-1, output_logits.size(-1))
            true_flat = targets[:, 1:].reshape(-1)  # Shift targets by one for teacher forcing

            # Compute loss and perform optimization
            loss = loss_fn(pred_flat, true_flat)
            loss.backward()
            
            # Apply gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optim.step()

            total_loss += loss.item()
                
            # Accuracy
            acc = compute_accuracy(output_logits, targets[:, 1:])
            total_acc += acc
            processed_batches += 1

        except Exception as err:
            print(f"Error in batch {i}/{total_batches}: {err}")
            continue

    if processed_batches == 0:
        return 0.0, 0.0
    return total_loss / processed_batches, total_acc / processed_batches

In [6]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    processed_batches = 0

    with torch.no_grad():
        for inputs, targets in data_loader:
            try:
                inputs = inputs.to(device)
                targets = targets.to(device)

                # Ensure token indices are within valid vocabulary range
                if inputs.max() >= model.input_embedding.num_embeddings or \
                   targets.max() >= model.output_embedding.num_embeddings:
                    continue  # Skip batch if invalid indices found

                # Forward propagation
                predictions = model(inputs, targets)

                # Flatten tensors for loss calculation
                pred_flat = predictions.reshape(-1, predictions.size(-1))
                target_flat = targets[:, 1:].reshape(-1)  # Shift targets by one for teacher forcing

                # Compute batch loss
                loss = loss_fn(pred_flat, target_flat)
                total_loss += loss.item()
                
                # Compute accuracy
                acc = compute_accuracy(predictions, targets[:, 1:])
                total_acc += acc
                processed_batches += 1

            except Exception as ex:
                print(f"Evaluation error: {ex}")
                continue

    if processed_batches == 0:
        return 0.0, 0.0
    return total_loss / processed_batches, total_acc / processed_batches

In [8]:
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'embed_dim': {'values': [16, 32, 64, 256]},
        'hidden_dim': {'values': [16, 32, 64, 256]},
        'cell_type': {'values': ['RNN','GRU','LSTM']},
        'enc_layers': {'values': [1, 2, 3]},
        'dec_layers': {'values': [1, 2, 3]},
        'dropout': {'values': [0.2, 0.3]},
        'learning_rate': {'values': [1e-3, 1e-4]},
        'batch_size': {'values': [32, 64]},
        'beam_size': {'values': [1, 3, 5]}
    }
}

In [7]:
def export_vocabularies(directory, source_dict, target_dict):
    """Export source and target vocabularies to JSON format."""
    os.makedirs(directory, exist_ok=True)
    
    src_path = os.path.join(directory, 'src.json')
    tgt_path = os.path.join(directory, 'tgt.json')
    
    with open(src_path, 'w', encoding='utf-8') as src_file:
        json.dump(source_dict, src_file, indent=2, ensure_ascii=False)
        
    with open(tgt_path, 'w', encoding='utf-8') as tgt_file:
        json.dump(target_dict, tgt_file, indent=2, ensure_ascii=False)

def import_vocabularies(directory):
    """Import source and target vocabularies from JSON files."""
    with open(os.path.join(directory, 'src.json'), 'r', encoding='utf-8') as src_file:
        source_dict = json.load(src_file)
        
    with open(os.path.join(directory, 'tgt.json'), 'r', encoding='utf-8') as tgt_file:
        target_dict = json.load(tgt_file)
        
    return source_dict, target_dict

In [8]:
# Dataset paths
path_train = '/kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.train.tsv'
path_dev = '/kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.dev.tsv'
vocab_dir = '/kaggle/working/vocab'

print("Generating vocabularies...")
train_data = TransliterationDataset(path_train, create_vocab=True)
src_vocab, tgt_vocab = train_data.source_vocab, train_data.target_vocab

export_vocabularies(vocab_dir, src_vocab, tgt_vocab)
print(f"Source vocab: {len(src_vocab)}, Target vocab: {len(tgt_vocab)}")

# Display sample vocab entries
print("Source vocab sample:")
for ch, idx in list(src_vocab.items())[:10]:
    print(f"  {repr(ch)} => {idx}")
print("Target vocab sample:")
for ch, idx in list(tgt_vocab.items())[:10]:
    print(f"  {repr(ch)} => {idx}")

Generating vocabularies...
Loaded 58550 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.train.tsv
Example pairs: [('amkita', 'అంకిత'), ('ankita', 'అంకిత')]
Vocab sizes — Source: 30, Target: 67
Source vocab: 30, Target vocab: 67
Source vocab sample:
  '<pad>' => 0
  '<unk>' => 1
  '<sos>' => 2
  '<eos>' => 3
  'a' => 4
  'm' => 5
  'k' => 6
  'i' => 7
  't' => 8
  'n' => 9
Target vocab sample:
  '<pad>' => 0
  '<unk>' => 1
  '<sos>' => 2
  '<eos>' => 3
  'అ' => 4
  'ం' => 5
  'క' => 6
  'ి' => 7
  'త' => 8
  'భ' => 9


In [11]:
def run_sweep():
    """Function to run for each sweep configuration"""
    run = wandb.init()
    cfg = run.config
        
    # Create a descriptive run name
    run.name = f"{cfg.cell_type}-e{cfg.embed_dim}-h{cfg.hidden_dim}-enc{cfg.enc_layers}-dec{cfg.dec_layers}-d{cfg.dropout}-lr{cfg.learning_rate}-b{cfg.batch_size}-beam{cfg.beam_size}"
        
    # Set device - force CPU initially if CUDA issues were detected
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    epochs = 20
    # Initialize model
    try:
        # First create on CPU
        model = Seq2Seq(cfg, len(src_vocab), len(tgt_vocab))
        print("Model created on CPU, trying to move to device...")
        # Then try to move to target device
        model = model.to(device)
        print("Model successfully moved to device.")
    except Exception as e:
        print(f"Error initializing model on {device}: {e}")
        print("Falling back to CPU")
        device = torch.device('cpu')
        model = Seq2Seq(cfg, len(src_vocab), len(tgt_vocab)).to(device)
        
    # Load datasets
    try:
        print("Loading datasets...")
        train_dataset = TransliterationDataset(path_train, src_vocab, tgt_vocab)
        dev_dataset = TransliterationDataset(path_dev, src_vocab, tgt_vocab)
            
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
        dev_loader = DataLoader(dev_dataset, batch_size=cfg.batch_size)
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return
        
    # Initialize loss function and optimizer
    try:
        criterion = nn.CrossEntropyLoss(ignore_index=0)  # 0 is padding index
        optimizer = optim.Adam(model.parameters(), lr=cfg.learning_rate)
    except Exception as e:
        print(f"Error initializing criterion or optimizer: {e}")
        return
        
    # Training loop
    best_val_acc = 0.0
        
    try:
        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
                
            # Training
            train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer, device)
            print(f"Train loss: {train_loss:.4f} Train Accuracy: {train_acc:.4f}")
                
            # Validation
            val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
            print(f"Validation loss: {val_loss:.4f} Validation Accuracy: {val_acc:.4f} ")
                
            # Log metrics
            wandb.log({'train_loss': train_loss, 'val_loss': val_loss, 'train_acc': train_acc, 'val_acc': val_acc, 'epoch': epoch})
                
            # Save best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
        wandb.log({'val_accuracy': best_val_acc})
                    
    except Exception as e:
        print(f"Error during training: {e}")

In [9]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")

In [10]:
wandb.login(key = secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m027 (da24m027-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [14]:
sweep_id = wandb.sweep(sweep_config, project='DA6401_Assignment3')

Create sweep with ID: zodbp4ie
Sweep URL: https://wandb.ai/da24m027-indian-institute-of-technology-madras/DA6401_Assignment3/sweeps/zodbp4ie


In [15]:
wandb.agent(sweep_id, run_sweep, count=50)

wandb: Agent Starting Run: 4ojq8wom with config:
wandb: 	batch_size: 64
wandb: 	beam_size: 1
wandb: 	cell_type: GRU
wandb: 	dec_layers: 1
wandb: 	dropout: 0.3
wandb: 	embed_dim: 16
wandb: 	enc_layers: 1
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.0001


Initialized model: GRU, Encoder layers: 1, Decoder layers: 1, Embedding: 16, Hidden: 256
Model created on CPU, trying to move to device...
Model successfully moved to device.
Loading datasets...
Loaded 58550 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.train.tsv
Example pairs: [('amkita', 'అంకిత'), ('ankita', 'అంకిత')]
Loaded 5683 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.dev.tsv
Example pairs: [('amka', 'అంక'), ('anka', 'అంక')]
Epoch 1/20
Train loss: 3.0276 Train Accuracy: 0.1912
Validation loss: 2.7446 Validation Accuracy: 0.2559 
Epoch 2/20
Train loss: 2.6103 Train Accuracy: 0.2740
Validation loss: 2.5035 Validation Accuracy: 0.3028 
Epoch 3/20
Train loss: 2.3186 Train Accuracy: 0.3466
Validation loss: 2.1728 Validation Accuracy: 0.3957 
Epoch 4/20
Train loss: 1.9679 Train Accuracy: 0.4436
Validation loss: 1.8296 Validation Accuracy: 0.4764 
Epoch 5/20
Train loss: 1.6606 Train Accuracy: 0.5248
Validation loss: 1.5603 Validatio

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_acc,▁▂▃▄▄▅▆▆▆▇▇▇▇▇██████
train_loss,█▇▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁
val_acc,▁▂▃▄▄▅▆▆▇▇▇▇▇███████
val_accuracy,▁
val_loss,█▇▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁
epoch,19
train_acc,0.88018
train_loss,0.41975
val_acc,0.83337
val_accuracy,0.83337


wandb: Agent Starting Run: je1gmagk with config:
wandb: 	batch_size: 32
wandb: 	beam_size: 5
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 2
wandb: 	dropout: 0.3
wandb: 	embed_dim: 256
wandb: 	enc_layers: 2
wandb: 	hidden_dim: 16
wandb: 	learning_rate: 0.001


Initialized model: LSTM, Encoder layers: 2, Decoder layers: 2, Embedding: 256, Hidden: 16
Model created on CPU, trying to move to device...
Model successfully moved to device.
Loading datasets...
Loaded 58550 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.train.tsv
Example pairs: [('amkita', 'అంకిత'), ('ankita', 'అంకిత')]
Loaded 5683 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.dev.tsv
Example pairs: [('amka', 'అంక'), ('anka', 'అంక')]
Epoch 1/20
Train loss: 2.8880 Train Accuracy: 0.1978
Validation loss: 2.6326 Validation Accuracy: 0.2614 
Epoch 2/20
Train loss: 2.5825 Train Accuracy: 0.2757
Validation loss: 2.5089 Validation Accuracy: 0.3009 
Epoch 3/20
Train loss: 2.5029 Train Accuracy: 0.2934
Validation loss: 2.4408 Validation Accuracy: 0.3132 
Epoch 4/20
Train loss: 2.4184 Train Accuracy: 0.3094
Validation loss: 2.3460 Validation Accuracy: 0.3362 
Epoch 5/20
Train loss: 2.3482 Train Accuracy: 0.3274
Validation loss: 2.2567 Validati

...

Initialized model: LSTM, Encoder layers: 3, Decoder layers: 3, Embedding: 64, Hidden: 256
Model created on CPU, trying to move to device...
Model successfully moved to device.
Loading datasets...
Loaded 58550 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.train.tsv
Example pairs: [('amkita', 'అంకిత'), ('ankita', 'అంకిత')]
Loaded 5683 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.dev.tsv
Example pairs: [('amka', 'అంక'), ('anka', 'అంక')]
Epoch 1/20
Train loss: 2.3052 Train Accuracy: 0.3511
Validation loss: 1.3998 Validation Accuracy: 0.5900 
Epoch 2/20
Train loss: 0.9245 Train Accuracy: 0.7232
Validation loss: 0.6254 Validation Accuracy: 0.8115 
Epoch 3/20
Train loss: 0.5081 Train Accuracy: 0.8449
Validation loss: 0.4403 Validation Accuracy: 0.8627 
Epoch 4/20
Train loss: 0.3665 Train Accuracy: 0.8867
Validation loss: 0.3928 Validation Accuracy: 0.8768 
Epoch 5/20
Train loss: 0.2906 Train Accuracy: 0.9098
Validation loss: 0.3568 Validati

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_acc,▁▅▇▇▇▇██████████████
train_loss,█▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆▇▇████████████████
val_accuracy,▁
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
train_acc,0.97638
train_loss,0.07191
val_acc,0.90431
val_accuracy,0.90572


wandb: Agent Starting Run: lcfmfw2y with config:
wandb: 	batch_size: 32
wandb: 	beam_size: 1
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 3
wandb: 	dropout: 0.2
wandb: 	embed_dim: 32
wandb: 	enc_layers: 3
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.001


Initialized model: LSTM, Encoder layers: 3, Decoder layers: 3, Embedding: 32, Hidden: 256
Model created on CPU, trying to move to device...
Model successfully moved to device.
Loading datasets...
Loaded 58550 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.train.tsv
Example pairs: [('amkita', 'అంకిత'), ('ankita', 'అంకిత')]
Loaded 5683 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.dev.tsv
Example pairs: [('amka', 'అంక'), ('anka', 'అంక')]
Epoch 1/20
Train loss: 2.0853 Train Accuracy: 0.4081
Validation loss: 1.2976 Validation Accuracy: 0.6174 
Epoch 2/20
Train loss: 0.8417 Train Accuracy: 0.7483
Validation loss: 0.5895 Validation Accuracy: 0.8200 
Epoch 3/20
Train loss: 0.4326 Train Accuracy: 0.8685
Validation loss: 0.4411 Validation Accuracy: 0.8615 
Epoch 4/20
Train loss: 0.2991 Train Accuracy: 0.9079
Validation loss: 0.4061 Validation Accuracy: 0.8772 
Epoch 5/20
Train loss: 0.2316 Train Accuracy: 0.9281
Validation loss: 0.3865 Validati

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_acc,▁▅▇▇▇███████████████
train_loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆▇▇████████████████
val_accuracy,▁
val_loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂
epoch,19
train_acc,0.97983
train_loss,0.06036
val_acc,0.89701
val_accuracy,0.89737


## Testing

In [11]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
import csv
from IPython.display import display, HTML

In [12]:
path_test = "/kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.test.tsv"

In [13]:
test_dataset = TransliterationDataset(path_test, src_vocab, tgt_vocab)
test_loader = DataLoader(test_dataset, batch_size=1)

Loaded 5747 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.test.tsv
Example pairs: [('amkamlo', 'అంకంలో'), ('ankamlo', 'అంకంలో')]


In [14]:
def decode_sequence(seq, char_vocab, end_token=3):
    """
    Convert index sequence to character sequence
    
    Args:
        seq: Sequence of indices
        char_vocab: Dictionary mapping characters to indices
        end_token: Token index representing end of sequence (default=2)
        
    Returns:
        String of characters
    """
    # Create inverse mapping (idx -> char)
    idx_to_char = {idx: ch for ch, idx in char_vocab.items()}
    
    # Process sequence and stop at end token
    result = []
    has_end_token = False
    
    for idx in seq:
        if idx == 0:  # Skip padding token
            continue
        if idx == end_token:  # Stop at end token
            has_end_token = True
            break
        if idx in idx_to_char:
            result.append(idx_to_char[idx])
        else:
            # Handle out-of-vocabulary tokens
            result.append('<UNK>')
    
    # If no end token was found, use the entire sequence
    # (this helps prevent truncated outputs)
    decoded = ''.join(result)
    
    return decoded

In [15]:
def evaluate_model(model, test_loader, src_vocab, tgt_vocab, device, end_token=3, unk_token=1, output_file="predictions_vanilla.csv"):
    """
    Evaluate model on test set and save results to CSV
    
    Args:
        model: The trained model
        test_loader: DataLoader for test dataset
        src_vocab: Source vocabulary (char -> idx mapping)
        tgt_vocab: Target vocabulary (char -> idx mapping)
        device: Device to run evaluation on
        end_token: Token index representing end of sequence (default=2)
        unk_token: Token index representing unknown token (default=1)
        output_file: Path to save CSV results
        
    Returns:
        List of results with input, prediction, target, and correctness
    """
    model.eval()
    
    results = []
    
    # Get target vocabulary size
    tgt_vocab_size = len(tgt_vocab)
    
    # Create inverse mapping for vocabulary inspection
    idx_to_char = {idx: ch for ch, idx in tgt_vocab.items()}
    print(f"Target vocabulary size: {len(tgt_vocab)}")
    print(f"Special tokens: PAD={0}, UNK={unk_token}, SOS={2}, EOS={end_token}")
    
    with torch.no_grad(), open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['input', 'prediction', 'target', 'correct'])
        
        for batch in tqdm(test_loader, desc="Evaluating"):
            src, tgt = batch
            src, tgt = src.to(device), tgt.to(device)
            
            # Forward pass without teacher forcing
            if hasattr(model, 'inference'):
                output = model.inference(src)
            else:
                # Use our custom inference function
                output = inference_without_teacher_forcing(
                    model, src, max_len=100, 
                    device=device
                )
            
            # Process each sequence in the batch
            for i in range(src.size(0)):
                src_seq = src[i].cpu().numpy()
                
                # If tgt starts with UNK token (used instead of SOS), skip it
                if tgt[i, 0].item() == unk_token:
                    tgt_seq = tgt[i, 1:].cpu().numpy()  # Skip UNK token
                else:
                    tgt_seq = tgt[i].cpu().numpy()  # Use entire sequence
                
                pred_seq = output[i].cpu().numpy()
                
                # Convert sequences to strings (exclude the sos in the beginning)
                src_str = decode_sequence(src_seq, src_vocab, end_token)
                tgt_str = decode_sequence(tgt_seq, tgt_vocab, end_token)
                pred_str = decode_sequence(pred_seq, tgt_vocab, end_token)
                
                # Check correctness
                correct = pred_str == tgt_str[5:]
                
                # Save to CSV
                writer.writerow([
                    src_str[5:], 
                    pred_str, 
                    tgt_str[5:]
                ])
                
                # Save for visualization
                results.append({
                    'input': src_str[5:],
                    'prediction': pred_str,
                    'target': tgt_str[5:],
                    'correct': correct
                })
    
    # Calculate overall accuracy
    accuracy = sum(1 for r in results if r['correct']) / len(results) if results else 0
    print(f"Overall Word Accuracy: {accuracy:.4f}")
    
    return accuracy, results

In [16]:
def generate_char_comparison_html(input_str, pred_str, target_str):
    """
    Generate HTML with character-by-character comparison
    """
    html = f"<p><b>Input:</b> {input_str}</p>"
    
    html += "<p><b>Prediction vs Target:</b> "
    
    # Extend the shorter string with spaces to match the longer one
    max_len = max(len(pred_str), len(target_str))
    pred_str_padded = pred_str.ljust(max_len)
    target_str_padded = target_str.ljust(max_len)
    
    for p_char, t_char in zip(pred_str_padded, target_str_padded):
        if p_char == t_char:
            html += f'<span style="color:green">{p_char}</span>'
        else:
            html += f'<span style="color:red">{p_char}</span>'
    
    html += "</p>"
    
    html += f"<p><b>Target:</b> {target_str}</p>"
    
    return html

In [17]:
def visualize_errors(results, n_samples=10):
    """
    Visualize character-level errors for random samples
    """
    # Sample random results (prioritize some incorrect ones)
    incorrect_samples = [r for r in results if not r['correct']]
    correct_samples = [r for r in results if r['correct']]
    
    if len(incorrect_samples) > 0 and len(correct_samples) > 0:
        # Try to get a mix of correct and incorrect samples
        n_incorrect = min(n_samples // 2, len(incorrect_samples))
        n_correct = n_samples - n_incorrect
        
        samples = (random.sample(incorrect_samples, n_incorrect) + 
                  random.sample(correct_samples, min(n_correct, len(correct_samples))))
    else:
        # If all samples are correct or incorrect, just sample randomly
        samples = random.sample(results, min(n_samples, len(results)))
    
    html_output = "<h2>Character-level Error Visualization</h2>"
    html_output += "<p>Green: Correct characters, Red: Incorrect characters</p>"
    
    for i, sample in enumerate(samples):
        html_output += f"<h3>Sample {i+1}</h3>"
        html_output += generate_char_comparison_html(
            sample['input'], 
            sample['prediction'], 
            sample['target']
        )
        html_output += "<hr>"
    
    return HTML(html_output)

In [18]:
api = wandb.Api()
sweep = api.sweep("da24m027-indian-institute-of-technology-madras/DA6401_Assignment3/zodbp4ie")
best_run = sweep.best_run()
best_run_config = best_run.config

wandb: Sorting runs by -summary_metrics.val_accuracy


In [19]:
best_run_config

{'dropout': 0.3,
 'beam_size': 3,
 'cell_type': 'LSTM',
 'embed_dim': 64,
 'batch_size': 64,
 'dec_layers': 3,
 'enc_layers': 3,
 'hidden_dim': 256,
 'learning_rate': 0.001}

In [20]:
def test_and_evaluate(config=None):
    run = wandb.init(config=config, project="DA6401_Assignment3")
    cfg = run.config
    epochs = 20
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    run.name = "Testing without Attention"

    # First create on CPU
    model = Seq2Seq(cfg, len(src_vocab), len(tgt_vocab))
    model = model.to(device)
    print("Model successfully moved to device.")
        
    print("Loading datasets...")
    train_dataset = TransliterationDataset(path_train, src_vocab, tgt_vocab)
    dev_dataset = TransliterationDataset(path_dev, src_vocab, tgt_vocab)
            
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
    dev_loader = DataLoader(dev_dataset, batch_size=cfg.batch_size)
        
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # 0 is padding index
    optimizer = optim.Adam(model.parameters(), lr=cfg.learning_rate)
    best_val_acc = 0.0
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
                
        # Training
        train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer, device)
        print(f"Train loss: {train_loss:.4f} Train Accuracy: {train_acc:.4f}")
                
        # Validation
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        print(f"Validation loss: {val_loss:.4f} Val Accuracy: {val_acc:.4f}")
                
        # Log metrics
        wandb.log({'train_loss': train_loss, 'val_loss': val_loss, 'train_acc': train_acc, 'val_acc': val_acc, 'epoch': epoch})
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'model.pth')
    test_acc, results = evaluate_model(model, test_loader, src_vocab, tgt_vocab, device)
    wandb.log({'Test_acc':test_acc})
    display(visualize_errors(results, n_samples=10))


In [21]:
test_and_evaluate(best_run_config)

Initialized model without attention: LSTM, Encoder layers: 3, Decoder layers: 3, Embedding: 64, Hidden: 256
Model successfully moved to device.
Loading datasets...
Loaded 58550 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.train.tsv
Example pairs: [('amkita', 'అంకిత'), ('ankita', 'అంకిత')]
Loaded 5683 records from /kaggle/input/dakshina-telugu/lexicons/te.translit.sampled.dev.tsv
Example pairs: [('amka', 'అంక'), ('anka', 'అంక')]
Epoch 1/20
Train loss: 2.3382 Train Accuracy: 0.3496
Validation loss: 1.4840 Val Accuracy: 0.5830
Epoch 2/20
Train loss: 0.8767 Train Accuracy: 0.7456
Validation loss: 0.4605 Val Accuracy: 0.8642
Epoch 3/20
Train loss: 0.3857 Train Accuracy: 0.8842
Validation loss: 0.3320 Val Accuracy: 0.8971
Epoch 4/20
Train loss: 0.2666 Train Accuracy: 0.9191
Validation loss: 0.3029 Val Accuracy: 0.9084
Epoch 5/20
Train loss: 0.2091 Train Accuracy: 0.9364
Validation loss: 0.2894 Val Accuracy: 0.9131
Epoch 6/20
Train loss: 0.1731 Train Accuracy: 0.947

Evaluating: 100%|██████████| 5747/5747 [00:35<00:00, 164.11it/s]

Overall Word Accuracy: 0.5453
